In [66]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display
import polars as pl
from datetime import timedelta
import datetime 
import json 
import toml
import holidays
import sys

features = toml.load(r'C:\Users\N000193384\Documents\sncf_project\sncf_playground\data\features.toml')
times_cols = features['times_cols']
macro_horizon = features['MACRO_HORIZON']
p = Path(features['ABS_DATA_PATH'])
sys.path.insert(1, p)

from src.preprocessing.times import (
    from_day_to_time_fe,
    get_covid_table,
)
from src.preprocessing.quality import trim_timeseries, minimum_length_uid
from src.models.forecast.direct import DirectForecaster
from src.preprocessing.lags import get_significant_lags
from src.preprocessing.times import get_basic_holidays
from src.project_utils import load_data
from src.models.lgb_wrapper import GBTModel
from src.preprocessing.validation import freeze_validation_set
from src.preprocessing.lags import compute_autoreg_features

ts_uid = features["ts_uid"]
date_col = features['date_col']
y = features['y']
submit = False 
flist = features["flist"]
long_horizon = np.arange(macro_horizon)
chains = np.array_split(long_horizon, 6)
exog = ["job", "ferie", "vacances"] + times_cols

with open('data/params.json', 'rb') as stream:
    params_q = json.load(stream)

in_dt = datetime.date(2020, 6, 1)

covid_df = get_covid_table(2015, 2024)
df_dates = df_dates = get_basic_holidays()
holidays_fe = list(filter(lambda x: date_col not in x, df_dates.columns))
covid_fe = list(filter(lambda x: date_col not in x, covid_df.columns))
exog = exog + holidays_fe + covid_fe

train_data, test_data, submission = load_data(p)

test_data = (
    test_data.pipe(from_day_to_time_fe, time=date_col, frequency="day")
    .join(df_dates, how="left", on=[date_col])
    .join(covid_df, how="left", on=[date_col])
)

train_data = (
    train_data.pipe(trim_timeseries, target="y", uid=ts_uid, time=date_col)
    .pipe(from_day_to_time_fe, time=date_col, frequency="day")
    .join(df_dates, how="left", on=[date_col])
    .join(
        covid_df.with_columns(
            pl.lit(np.where(np.any(covid_df != 0, axis=1), 0.01, 1)).alias(
                "covid_weight"
            )
        ),
        how="left",
        on=[date_col],
    )
    # sncf strike | 2019-12-01 to 2021-11-01
    .filter(
        (pl.col(date_col) >= in_dt)
        & (pl.col(date_col) != pl.datetime(2019, 12, 1))
        & (pl.col(date_col) != pl.datetime(2021, 11, 1))
    )
)

good_ts = minimum_length_uid(
    train_data, target=y, uid=ts_uid, time=date_col, min_length=round(364 * 1.2)
)
train_data = train_data.filter(pl.col(ts_uid).is_in(good_ts))
# test.
left_term = train_data.select([date_col, ts_uid, "y"] + exog).with_columns(
    pl.lit(1).alias("train")
)
right_term = test_data.select([date_col, ts_uid, "y"] + exog).with_columns(
    pl.lit(0).alias("train")
)
full_data = pl.concat((left_term, right_term), how="vertical_relaxed")
del left_term, right_term

# define params
significant_lags = get_significant_lags(train_data, date_col=date_col, target=y)
significant_lags = [x for x in significant_lags if x%7 == 0 and x<= macro_horizon][1:]

In [51]:
autoreg_dict = {
        ts_uid: {
            "groups": ts_uid,
            "horizon": lambda horizon: np.int32(horizon),
            "wins": np.array([28, 56]),
            "shifts": lambda horizon: np.int32([horizon]),
            "lags": lambda horizon: np.array(significant_lags) + horizon,
            "funcs": np.array(flist),
        },
        "ts_uid_dow": {
            "groups": [ts_uid, "day_of_week"],
            "horizon": lambda horizon: np.int32(np.ceil(horizon / 7) + 1),
            "wins": np.array([4, 8]),
            "shifts": lambda horizon: np.int32([np.ceil(horizon / 7) + 1]),
            "lags": lambda horizon: np.arange(1, 7) + np.ceil(horizon / 7) + 1,
            "funcs": np.array(flist),
        }
    }

for key in autoreg_dict.keys():
    autoreg_dict[key]["horizon"] = autoreg_dict[key]["horizon"](macro_horizon)
    autoreg_dict[key]["shifts"] = autoreg_dict[key]["shifts"](macro_horizon)
    autoreg_dict[key]["lags"] = autoreg_dict[key]["lags"](macro_horizon)

train_set, val_set = freeze_validation_set(
    ((train_data
     .with_columns(pl.col('y').log1p().alias('y'))
    )
    .pipe(compute_autoreg_features, 
           target_col="y", 
           date_str=date_col, 
           auto_reg_params=autoreg_dict
           )
    .fill_null(0)
    .fill_nan(0)
    .filter(pl.col(date_col) >=  datetime.datetime(2019, 1, 1))
    ),
    date=date_col, 
    val_size=181, 
    return_train=True
    )


num_cols =  ["job", "ferie", "vacances"] + holidays_fe + list(filter(lambda x : x.startswith('ar_'), train_set.columns))
cat_cols = ['week', 'month', 'day_of_week', 'day']
target = "y"

from sklearn.preprocessing import StandardScaler
sc = StandardScaler().set_output(transform='polars')
# Fit and transform the selected columns
scaled_values = sc.fit_transform(train_set.select(num_cols))
scaled_test_values = sc.transform(val_set.select(num_cols))
# Add the scaled data as new columns to the Polars DataFrame
for i, col in enumerate(num_cols):
    train_set = train_set.with_columns(pl.Series(f"scaled_{col}", scaled_values[col]))
    val_set = val_set.with_columns(pl.Series(f"scaled_{col}", scaled_test_values[col]))

scaled_num_cols = [f"scaled_{c}" for c in num_cols]
display(train_set.null_count(), val_set.null_count())

date,station,job,ferie,vacances,index,is_train,y,y_copy,min_dt,year,week,month,day,day_of_year,day_of_week,quarter,week_of_month,is_weekend,Jour de l'an,Lundi de Pâques,Fête du Travail,Fête de la Victoire,Ascension,Lundi de Pentecôte,Fête nationale,Assomption,Toussaint,Armistice,Noël,First Lockdown,Second Lockdown,Third Lockdown (Partial),covid_weight,ar_y_lag188_station,ar_y_lag195_station,ar_y_lag202_station,…,scaled_Fête du Travail,scaled_Fête de la Victoire,scaled_Ascension,scaled_Lundi de Pentecôte,scaled_Fête nationale,scaled_Assomption,scaled_Toussaint,scaled_Armistice,scaled_Noël,scaled_ar_y_lag188_station,scaled_ar_y_lag195_station,scaled_ar_y_lag202_station,scaled_ar_y_lag230_station,scaled_ar_y_lag251_station,scaled_ar_y_lag300_station,scaled_ar_y_win28_shift181_rolling_mean_station,scaled_ar_y_win28_shift181_rolling_median_station,scaled_ar_y_win28_shift181_rolling_std_station,scaled_ar_y_win28_shift181_rolling_skew_station,scaled_ar_y_win56_shift181_rolling_mean_station,scaled_ar_y_win56_shift181_rolling_median_station,scaled_ar_y_win56_shift181_rolling_std_station,scaled_ar_y_win56_shift181_rolling_skew_station,scaled_ar_y_lag28.0_station@day_of_week,scaled_ar_y_lag29.0_station@day_of_week,scaled_ar_y_lag30.0_station@day_of_week,scaled_ar_y_lag31.0_station@day_of_week,scaled_ar_y_lag32.0_station@day_of_week,scaled_ar_y_lag33.0_station@day_of_week,scaled_ar_y_win4_shift27_rolling_mean_station@day_of_week,scaled_ar_y_win4_shift27_rolling_median_station@day_of_week,scaled_ar_y_win4_shift27_rolling_std_station@day_of_week,scaled_ar_y_win4_shift27_rolling_skew_station@day_of_week,scaled_ar_y_win8_shift27_rolling_mean_station@day_of_week,scaled_ar_y_win8_shift27_rolling_median_station@day_of_week,scaled_ar_y_win8_shift27_rolling_std_station@day_of_week,scaled_ar_y_win8_shift27_rolling_skew_station@day_of_week
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


date,station,job,ferie,vacances,index,is_train,y,y_copy,min_dt,year,week,month,day,day_of_year,day_of_week,quarter,week_of_month,is_weekend,Jour de l'an,Lundi de Pâques,Fête du Travail,Fête de la Victoire,Ascension,Lundi de Pentecôte,Fête nationale,Assomption,Toussaint,Armistice,Noël,First Lockdown,Second Lockdown,Third Lockdown (Partial),covid_weight,ar_y_lag188_station,ar_y_lag195_station,ar_y_lag202_station,…,scaled_Fête du Travail,scaled_Fête de la Victoire,scaled_Ascension,scaled_Lundi de Pentecôte,scaled_Fête nationale,scaled_Assomption,scaled_Toussaint,scaled_Armistice,scaled_Noël,scaled_ar_y_lag188_station,scaled_ar_y_lag195_station,scaled_ar_y_lag202_station,scaled_ar_y_lag230_station,scaled_ar_y_lag251_station,scaled_ar_y_lag300_station,scaled_ar_y_win28_shift181_rolling_mean_station,scaled_ar_y_win28_shift181_rolling_median_station,scaled_ar_y_win28_shift181_rolling_std_station,scaled_ar_y_win28_shift181_rolling_skew_station,scaled_ar_y_win56_shift181_rolling_mean_station,scaled_ar_y_win56_shift181_rolling_median_station,scaled_ar_y_win56_shift181_rolling_std_station,scaled_ar_y_win56_shift181_rolling_skew_station,scaled_ar_y_lag28.0_station@day_of_week,scaled_ar_y_lag29.0_station@day_of_week,scaled_ar_y_lag30.0_station@day_of_week,scaled_ar_y_lag31.0_station@day_of_week,scaled_ar_y_lag32.0_station@day_of_week,scaled_ar_y_lag33.0_station@day_of_week,scaled_ar_y_win4_shift27_rolling_mean_station@day_of_week,scaled_ar_y_win4_shift27_rolling_median_station@day_of_week,scaled_ar_y_win4_shift27_rolling_std_station@day_of_week,scaled_ar_y_win4_shift27_rolling_skew_station@day_of_week,scaled_ar_y_win8_shift27_rolling_mean_station@day_of_week,scaled_ar_y_win8_shift27_rolling_median_station@day_of_week,scaled_ar_y_win8_shift27_rolling_std_station@day_of_week,scaled_ar_y_win8_shift27_rolling_skew_station@day_of_week
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from typing import List
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

class TimeSeriesDataset(Dataset):
    def __init__(self, dataset:pl.DataFrame, continuous_data:List[str], categorical_data:List[str], target:str):
        self.continuous_data = dataset.select(continuous_data).to_numpy()
        self.categorical_data = dataset.select(categorical_data).to_numpy()
        self.targets = dataset.select(target).to_numpy().ravel()
            
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return {
            'x': torch.tensor(self.continuous_data[idx], dtype=torch.float32),
            'categorical_features': torch.tensor(self.categorical_data[idx], dtype=torch.long),
            'y': torch.tensor(self.targets[idx], dtype=torch.float32)
        }

class SmoothQLoss(nn.L1Loss):
    constants = ['reduction', 'beta', 'qx']
    def __str__(self):
        return f"{self.__class__.__name__}(beta={self.beta}, qx={self.qx})"
    
    def __init__(
        self,
        size_average=None,
        reduce=None,
        reduction: str = 'mean',
        qx: float = 0.5,
        beta: float = 0.1,
    ):
        super().__init__(size_average, reduce, reduction)
        self.beta = beta
        self.qx = qx
    
    def forward(
      self, 
      predict: torch.Tensor, target: torch.Tensor
    ) -> torch.Tensor:
        m = 2.0 * self.qx - 1.0
        shift = self.beta * m
        if self.reduction == 'mean':
            return (
                F.smooth_l1_loss(
                  target - shift, predict, reduction='mean', 
                  beta=self.beta
                ) + m * torch.mean(target - predict - 0.5 * shift)
            )
        elif self.reduction == 'sum':
            return (
                F.smooth_l1_loss(
                    target - shift, predict, reduction='sum', 
                    beta=self.beta
                ) + m * torch.sum(target - predict - 0.5 * shift)
            )
        
class TimeSeriesMLP(nn.Module):
    def __init__(self, num_features, embedding_sizes, hidden_dim=64, output_dim=1):
        super(TimeSeriesMLP, self).__init__()
        self.output_dim = output_dim
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories, size in embedding_sizes])
        embedding_dim = sum(e.embedding_dim for e in self.embeddings)
        input_dim = num_features + embedding_dim
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)

    def forward(self, x, categorical_features):
        embedded = [embedding(categorical_features[:, i]) for i, embedding in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        x = torch.cat([x, embedded], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.squeeze(x) if self.output_dim == 1 else x

train_ds = TimeSeriesDataset(train_set, 
    continuous_data=num_cols, 
    categorical_data=cat_cols, 
    target=target)

val_ds = TimeSeriesDataset(val_set, 
    continuous_data=num_cols, 
    categorical_data=cat_cols, 
    target=target)

# Create dataloaders
batch_size = 16
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)

# Example usage:
num_lags = len(num_cols)
num_categorical_features = len(cat_cols)
unique_modalities = {col: train_data[col].n_unique() + 1 for col in cat_cols}
embedding_sizes = [(nmodality, max(nmodality//4, 1)) for nmodality in unique_modalities.values()]
model = TimeSeriesMLP(num_features=num_lags, embedding_sizes=embedding_sizes)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, patience=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    patience_counter = 0

    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            x = batch['x']
            y = batch['y']
            categorical_features = batch['categorical_features']
            optimizer.zero_grad()
            outputs = model(x, categorical_features)
            loss = criterion(outputs.squeeze(), y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * x.size(0)

        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                x = batch['x']
                y = batch['y']
                categorical_features = batch['categorical_features']
                outputs = model(x, categorical_features)
                loss = criterion(outputs.squeeze(), y)
                val_loss += loss.item() * x.size(0)

        val_loss /= len(val_loader.dataset)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping")
            break

    model.load_state_dict(best_model_wts)
    return model

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model with early stopping
model = train_model(model, train_loader, val_loader, criterion, optimizer)

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:54<1:29:39, 54.34s/it]

Epoch 1/100, Train Loss: 0.8006, Val Loss: 0.3884


  2%|▏         | 2/100 [01:48<1:28:54, 54.43s/it]

Epoch 2/100, Train Loss: 0.5862, Val Loss: 0.3652


...

Epoch 20/100, Train Loss: 0.5275, Val Loss: 0.3761


 21%|██        | 21/100 [19:44<1:19:11, 60.14s/it]

Epoch 21/100, Train Loss: 0.5268, Val Loss: 0.4447


 21%|██        | 21/100 [20:44<1:18:02, 59.28s/it]

Epoch 22/100, Train Loss: 0.5260, Val Loss: 0.3726
Early stopping


In [65]:
def predict(model, x_test):
    x_test = TimeSeriesDataset(
            x_test,
            continuous_data=num_cols,
            categorical_data=cat_cols,
            target=target,
        )
    forecast_loader = DataLoader(x_test, batch_size=1, shuffle=False)
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in forecast_loader:
            x = batch["x"]
            categorical_features = batch["categorical_features"]
            outputs = model(x, categorical_features)
            predictions.append(outputs.item())
    return predictions

test_out = (full_data
            .with_columns(pl.col('y').log1p().alias('y'))
              .pipe(compute_autoreg_features, 
           target_col="y", 
           date_str=date_col, 
           auto_reg_params=autoreg_dict
           )
    .fill_null(0)
    .fill_nan(0)
    .filter(pl.col('train') == 0)
)
output = test_out.with_columns(pl.lit(np.expm1(predict(model, test_out))).alias('y_hat'))
output

date,station,y,job,ferie,vacances,week,month,day,day_of_week,quarter,week_of_month,is_weekend,Jour de l'an,Lundi de Pâques,Fête du Travail,Fête de la Victoire,Ascension,Lundi de Pentecôte,Fête nationale,Assomption,Toussaint,Armistice,Noël,First Lockdown,Second Lockdown,Third Lockdown (Partial),train,ar_y_lag188_station,ar_y_lag195_station,ar_y_lag202_station,ar_y_lag230_station,ar_y_lag251_station,ar_y_lag300_station,ar_y_win28_shift181_rolling_mean_station,ar_y_win28_shift181_rolling_median_station,ar_y_win28_shift181_rolling_std_station,ar_y_win28_shift181_rolling_skew_station,ar_y_win56_shift181_rolling_mean_station,ar_y_win56_shift181_rolling_median_station,ar_y_win56_shift181_rolling_std_station,ar_y_win56_shift181_rolling_skew_station,ar_y_lag28.0_station@day_of_week,ar_y_lag29.0_station@day_of_week,ar_y_lag30.0_station@day_of_week,ar_y_lag31.0_station@day_of_week,ar_y_lag32.0_station@day_of_week,ar_y_lag33.0_station@day_of_week,ar_y_win4_shift27_rolling_mean_station@day_of_week,ar_y_win4_shift27_rolling_median_station@day_of_week,ar_y_win4_shift27_rolling_std_station@day_of_week,ar_y_win4_shift27_rolling_skew_station@day_of_week,ar_y_win8_shift27_rolling_mean_station@day_of_week,ar_y_win8_shift27_rolling_median_station@day_of_week,ar_y_win8_shift27_rolling_std_station@day_of_week,ar_y_win8_shift27_rolling_skew_station@day_of_week,y_hat
date,str,f64,i64,i64,i64,i16,i16,i16,i16,i16,i16,i16,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-06-12,"""003""",0.0,1,0,0,24,6,12,1,2,2,0,-162,-55,-42,-35,-17,-6,32,64,142,152,-169,0.0,0.0,0.0,0,6.50279,6.444131,6.523562,6.190315,6.327937,5.669881,6.123243,6.3926,0.503429,-1.157499,5.970889,6.202462,0.539507,-0.735262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,571.546435
2023-01-16,"""003""",0.0,1,0,0,3,1,16,1,1,3,0,-15,84,105,112,122,133,179,-154,-76,-66,-22,0.0,0.0,0.0,0,5.958425,6.202536,6.302619,6.380123,6.415097,6.378426,5.491885,5.700393,0.780794,-2.109293,5.698871,5.812636,0.676915,-2.03004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,283.179948
2023-04-17,"""003""",0.0,1,0,0,16,4,17,1,2,3,0,-106,1,14,21,31,42,88,120,-167,-157,-113,0.0,0.0,0.0,0,6.383507,6.327937,6.052089,6.047372,5.497168,6.388561,5.867098,6.042588,0.526752,-0.852274,5.690566,5.922917,0.68685,-1.996619,0.0,0.0,0.0,0.0,0.0,0.0,5.762051,5.762051,0.0,0.0,5.762051,5.762051,0.0,0.0,301.04559
2023-02-13,"""003""",0.0,1,0,0,7,2,13,1,1,2,0,-43,56,77,84,94,105,151,-182,-104,-94,-50,0.0,0.0,0.0,0,5.497168,5.799093,5.937536,6.302619,6.455199,6.403574,5.478854,5.601294,0.354124,-0.572389,5.485369,5.655937,0.600735,-2.288161,6.418365,5.762051,0.0,0.0,0.0,0.0,6.003121,5.828946,0.361164,0.679914,6.003121,5.828946,0.361164,0.679914,434.628933
2023-05-08,"""003""",0.0,1,1,1,19,5,8,1,2,2,0,-127,-20,-7,0,10,21,67,99,177,-178,-134,0.0,0.0,0.0,0,5.153292,6.190315,5.720312,5.929589,6.047372,5.958425,5.832029,6.060998,0.476577,-0.525255,5.824091,5.995145,0.511528,-0.749416,5.817111,0.0,5.828946,6.418365,5.762051,0.0,0.0,0.0,0.0,0.0,0.0,6.123655,0.0,0.0,58.744744
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-03-19,"""ZXY""",0.0,0,0,0,11,3,19,7,1,3,1,-77,22,43,50,60,71,117,149,-138,-128,-84,0.0,0.0,0.0,0,8.8371,8.742893,8.412277,8.231642,8.534444,8.736489,8.319894,8.476619,0.561147,-0.942057,8.194059,8.260742,0.473778,-0.552283,0.0,0.0,0.0,7.577634,7.667626,6.269096,0.0,0.0,0.0,0.0,0.0,7.842137,0.0,0.0,445.725986
2023-02-05,"""ZXY""",0.0,0,0,0,5,2,5,7,1,1,1,-35,64,85,92,102,113,159,-174,-96,-86,-42,0.0,0.0,0.0,0,8.231642,8.453401,8.563313,8.67163,8.724207,8.74767,8.104626,8.307072,0.498925,-1.512387,8.22353,8.414379,0.490266,-1.322743,7.089243,0.0,0.0,0.0,7.577634,7.667626,0.0,0.0,0.0,0.0,0.0,7.62263,0.0,0.0,236.889672
2023-04-23,"""ZXY""",0.0,0,0,1,16,4,23,7,2,4,1,-112,-5,8,15,25,36,82,114,-173,-163,-119,0.

In [3]:
'''import os
from typing import Dict, List

import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


def objective(
    trial: optuna.trial,
    train_data: pd.DataFrame,
    exog: List,
    seed: int = 12345,
):
    """_summary_

    Args:
        trial (optuna.trial): _description_
        train_x (pd.DataFrame): _description_
        test (pd.DataFrame): _description_
        features (List): _description_
        target (str, optional): _description_. Defaults to "".
        seed (int, optional): _description_. Defaults to 12345.

    Returns:
        _type_: _description_
    """
    optuna_params = {
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "objective": trial.suggest_categorical(
            "objective", ["regression", "huber", "regression_l1", "quantile"]
        ),
        "metric": trial.suggest_categorical("metric", ["rmse"]),
        "alpha": trial.suggest_categorical(
            "alpha",
            [0.5, 0.52, 0.55, 0.57, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.7, 0.69, 0.7],
        ),
        "force_row_wise": trial.suggest_categorical("force_row_wise", [True, False]),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05, log=False),
        "max_depth": trial.suggest_int("max_depth", 4, 15),
        "sub_row": trial.suggest_categorical("sub_row", [0.6, 0.7, 0.8, 1.0]),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 4, log=True),
        "min_child_samples": trial.suggest_float(
            "min_child_samples", 20, 5000, log=False
        ),
        "num_iterations": trial.suggest_int(
            "n_estimators",
            200,
            3000,
        ),
        "num_leaves": trial.suggest_int("num_leaves", 25, 800),
        "max_bins": trial.suggest_int("max_bins", 24, 1000),
        "min_data_in_bin": trial.suggest_int("min_data_in_bin", 25, 1000),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 1000),
        "feature_fraction_seed": trial.suggest_categorical(
            "feature_fraction_seed", [seed]
        ),
        "bagging_seed": trial.suggest_categorical("bagging_seed", [seed]),
        "seed": trial.suggest_categorical("seed", [seed]),
        "verbose": trial.suggest_categorical("verbose", [-1]),
    }

    horizon = 181
    lags = [x for x in significant_lags if x <= horizon]
    win_list =  [ x for x in significant_lags if x % 7 == 0 and x <= horizon][1:] 

    autoreg_dict = {
        ts_uid : {
            'groups' : ts_uid,
            'horizon': lambda horizon : np.int32(horizon),
            'wins' : np.array(win_list), 
            'shifts' : lambda horizon : np.int32([horizon, horizon+28, horizon+56]), 
            'lags' : lambda horizon : np.array(significant_lags) + horizon,
            'funcs' : np.array(flist)
        },
        "ts_uid_dow" : {
            'groups':[ts_uid, 'day_of_week'],
            'horizon' : lambda horizon : np.int32(np.ceil(horizon /7) + 1),
            'wins' : np.array([4, 8, 12, 16, 20]), 
            'shifts' : lambda horizon : np.int32([np.ceil(horizon /7) + 1, np.ceil(horizon /7) + 4]), 
            'lags' : lambda horizon : np.arange(1, 7) + np.ceil(horizon/7)+1,
            'funcs' : np.array(flist)
        }
    }


    effect_m = GBTModel(params=optuna_params, 
                        early_stopping_value=200, 
                        features= None,
                        custom_loss=optuna_params["objective"], 
                        categorical_features=[]
                        )


    dir_forecaster = DirectForecaster(
        model=effect_m,
        ts_uid=ts_uid,
        forecast_range=np.arange(horizon),
        target_str="y",
        date_str="date",
        exogs=exog,
        features_params=autoreg_dict
    )

    dir_forecaster.fit(train_data=train_data)
    return dir_forecaster.evaluate()["mae"].values[0]


def parameters_tuning(
    initial_params: Dict,
    tuning_objective,
    n_trials: int = 25,
    njobs: int = -1,
):
    """parameter for tuning over sudy

    Args:
        tuning_objective (_type_): _description_
        n_trials (int, optional): _description_. Defaults to 25.

    Returns:
        _type_: _description_

    example :

    func = lambda trial: objective(trial=trial,
                                    train_x=train_x,
                                    test=residualised_test,
                                    covariates=covariates,
                                    target=y,
                                    seed=12345
                                    )
    study_df, best_params = parameters_tuning(tuning_objective=func, n_trials=25, initial_params={})
        print(best_params)
        print(study_df)
        study_df.to_csv('bparamslgb_new.csv', sep="|", index=False)
    """
    study = optuna.create_study(direction="minimize")
    # study.enqueue_trial(initial_params)
    study.optimize(tuning_objective, n_trials=n_trials, n_jobs=njobs)
    print("Number of finished trials:", len(study.trials))
    print("Best trial:", study.best_trial.params)
    study_df = study.trials_dataframe()
    return study_df, study.best_params


func = lambda trial: objective(trial=trial,
                                train_data=train_data,
                                exog=exog,
                                seed=12345
                                )
study_df, best_params = parameters_tuning(tuning_objective=func, n_trials=30, initial_params={}, njobs=10)
print(best_params)
print(study_df)
study_df.to_csv('bparamslgb_new.csv', sep="|", index=False)'''

'import os\nfrom typing import Dict, List\n\nimport lightgbm as lgb\nimport numpy as np\nimport optuna\nimport pandas as pd\nfrom sklearn.metrics import mean_squared_error\nfrom sklearn.model_selection import train_test_split\n\n\ndef objective(\n    trial: optuna.trial,\n    train_data: pd.DataFrame,\n    exog: List,\n    seed: int = 12345,\n):\n    """_summary_\n\n    Args:\n        trial (optuna.trial): _description_\n        train_x (pd.DataFrame): _description_\n        test (pd.DataFrame): _description_\n        features (List): _description_\n        target (str, optional): _description_. Defaults to "".\n        seed (int, optional): _description_. Defaults to 12345.\n\n    Returns:\n        _type_: _description_\n    """\n    optuna_params = {\n        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),\n        "objective": trial.suggest_categorical(\n            "objective", ["regression", "huber", "regression_l1", "quantile"]\n        ),\n        

In [ ]:
from neuralforecast.core import NeuralForecast
from neuralforecast.losses.pytorch import MAE, MSE
from neuralforecast.models import PatchTST, TSMixer, iTransformer
from typing import List, Union, Dict, Tuple
from src.analysis.metrics import display_metrics


class NeuralWrapper:

    def __init__(
        self,
        models: List,
        ts_uid: str,
        date_col: str,
        target: str,
        forecast_horizon: int,
        fill_strategy: str = "forward",
        frequency: str = "1d",
        levels: List[int] = [95],
        conformalised: bool = False,
        fitted: bool = False,
    ):
        self.models = models
        self.fill_strategy = fill_strategy
        self.ts_uid = ts_uid
        self.forecast_horizon = forecast_horizon
        self.date_col = date_col
        self.target = target
        self.frequency = frequency
        self.conformalised = conformalised
        self.levels = levels
        self.fitted = fitted
        self.forecast_range = np.arange(self.forecast_horizon)

    def fill_gap(self, x: pl.DataFrame, date_range):
        return (
            pl.DataFrame({self.ts_uid: x[self.ts_uid][0], self.date_col: date_range})
            .join(x, on=self.date_col, how="left")[
                [self.ts_uid, self.target, self.date_col]
            ]
            .with_columns(pl.col(self.target).fill_null(strategy=self.fill_strategy))
        )

    def nixtla_reformat(self, Y_df: pl.DataFrame, date_col: str) -> pl.DataFrame:
        # get min,  max date
        mn, mx = (
            Y_df[date_col].cast(pl.Date).min(),
            Y_df[date_col].cast(pl.Date).max(),
        )
        # construct the range
        r = pl.date_range(mn, mx, self.frequency, eager=True)
        # group by "id" and fill the missing dates
        Y_df = (
            Y_df.group_by(self.ts_uid, maintain_order=True)
            .apply(lambda x: self.fill_gap(x, r))
            .rename({date_col: "ds", self.ts_uid: "unique_id", self.target: "y"})
            .with_columns(pl.col("ds").cast(pl.Date).alias("ds"))
        )
        return Y_df

    def ensemble(self, forecasts_df):
        all_col = forecasts_df.columns
        lb_cols = list(filter(lambda x: "-lo-" in x, all_col))
        ub_cols = list(filter(lambda x: "-hi-" in x, all_col))
        point_fcst_cols = list(
            filter(lambda x: x not in ["ds", "unique_id"] + lb_cols + ub_cols, all_col)
        )
        if len(point_fcst_cols) > 1:
            forecasts_df = forecasts_df.with_columns(
                [
                    pl.concat_list(point_fcst_cols)
                    .list.mean()
                    .alias("arithmetic_forecast_ensamble"),
                    pl.concat_list(lb_cols)
                    .list.mean()
                    .alias("arithmetic_lower_bound_ensamble"),
                    pl.concat_list(ub_cols)
                    .list.mean()
                    .alias("arithmetic_upper_bound_ensamble"),
                ]
            )
        self.point_fcst_cols = point_fcst_cols
        return forecasts_df

    def temporal_train_test_split(
        self, data: pl.DataFrame, date_col: str
    ) -> Tuple[pl.DataFrame, pl.DataFrame]:
        date_series = data[date_col].cast(pl.Date)
        max_available_date = date_series.max()
        self.min_available_date = date_series.min()
        self.start_valid = max_available_date - timedelta(
            days=int(max(self.forecast_range))
        )
        self.end_valid = max_available_date
        # retrieve max date from data
        train = data.filter(pl.col(date_col) < self.start_valid)
        valid = data.filter(
            pl.col(date_col).is_between(self.start_valid, self.end_valid, closed="both")
        )
        return train, valid
    
    def fit(self, Y_df, val_size:int):
        self.models.fit(Y_df, val_size=val_size)

    def forecast(self):
        forecasts_df = (
            self.models.predict(
                fitted=self.fitted,
                level=self.levels,
                h=self.forecast_horizon,
            )
            .pipe(self.ensemble)
        )
        return forecasts_df

    def evaluate_on_valid(self, Y_df):
        train, valid = self.temporal_train_test_split(Y_df, date_col="ds")
        forecast_valid = self.forecast(self.fit(train))
        metrics = pl.DataFrame()
        for col in self.point_fcst_cols:
            metrics.append(display_metrics(valid[self.target], forecast_valid[col]))
        return valid, metrics


sf =NeuralForecast([
    PatchTST(h=macro_horizon, input_size=120, max_steps=1000, early_stop_patience_steps=3),
    TSMixer(h=macro_horizon, input_size=120, n_series=439, max_steps=1000, early_stop_patience_steps=3),
    iTransformer(h=macro_horizon, input_size=120, n_series=439, max_steps=1000, early_stop_patience_steps=3),
], freq="1d")

nf_base = NeuralWrapper(
    models=sf,
    ts_uid=ts_uid,
    date_col=date_col,
    target=y,
    forecast_horizon=macro_horizon,
    fill_strategy="forward",
    frequency="1d",
    levels=[95],
    conformalised=False,
    fitted=False)


train_data, test_data, submission = load_data(p)
train_data = (
    train_data.pipe(trim_timeseries, target="y", uid="station", time="date")
    .pipe(from_day_to_time_fe, time="date", frequency="day")
    .filter(
        (pl.col("date") >= datetime.datetime(2017, 1, 1))
        & (pl.col("date") != pl.datetime(2019, 12, 1))
        & (pl.col("date") != pl.datetime(2021, 11, 1))
    )
    .pipe(nf_base.nixtla_reformat, date_col="date")
    .drop_nulls()
)

nf_base.fit(train_data, val_size=macro_horizon)

In [ ]:
nf_output = nf_base.models.predict()
nf_output = (nf_output.with_columns(
    pl.concat_list(list(filter(lambda x : x not in ['unique_id', 'ds'], nf_output.columns))).list.mean().alias('ensamble_transformer'),
    pl.col('ds').cast(pl.Date).alias("date")
).rename({"unique_id":"station"})
.select(["date", "station", "ensamble_transformer"])
)

test_data_out = (test_data
 .with_columns(pl.col('date').cast(pl.Date))
 .join(nf_output, how="left", on=["date", "station"])
).drop('y').rename({"ensamble_transformer":"y"}).select('index', 'y')

test_data_out.write_csv('out/submit/neural_model_fcst.csv')

# evaluate

In [73]:
base_col = [ts_uid, date_col, y, "day_of_year"]

from src.preprocessing.lags import reference_shift_from_day
from src.analysis.metrics import display_metrics

def freeze_validation_set(
    df: pl.DataFrame,
    date: str,
    val_size: int,
    return_train: bool = True,
) -> pl.DataFrame:
    max_dt = df[date].max()
    cut = max_dt - timedelta(days=val_size)
    valid = df.filter(pl.col(date) > cut)  # .select([ts_uid, date, target])
    if return_train:
        train = df.filter(pl.col(date) <= cut)
        return train, valid
    else:
        return valid


"""statsmodel_valid = (pl.read_csv('out/nixtla_validation.csv', separator=",", infer_schema_length=1000)
                    .rename({"unique_id":ts_uid, "ds":date_col})
                    .select([ts_uid, date_col, 'HoltWinters', 'AutoETS', "AutoARIMA",
                              'AutoTheta', 'AutoTBATS', 'arithmetic_forecast_ensamble']
                              ).with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
)
"""

direct_global_valid = (pl.read_csv('out/global_direct_lgb.csv', separator=",", infer_schema_length=1000)
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
)

# neural_fcst_model = (pl.read_csv('out/global_direct_lgb.csv', separator=",", infer_schema_length=1000)
#                      .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
# )


direct_local_valid = (pl.read_csv('out/local_direct_lgb.csv', separator=",", infer_schema_length=1000)
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
)


chain_global_valid = (pl.read_csv('out/global_chain_lgb.csv', separator=",", infer_schema_length=1000)
                     # .rename({"y_hat":"global_chain_y_hat"})
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
)

baseline = (train_data
    .pipe(from_day_to_time_fe, time="date", frequency="day")
    .pipe(reference_shift_from_day,    
        target_col=y, 
        ts_uid=ts_uid,
        dayofyear_col="day_of_year"       )
    .pipe(freeze_validation_set, return_train=False, date=date_col, val_size=181)
    .with_columns(
        pl.coalesce(
            pl.col('reference_y'), 
            pl.col('reference_y').mean().over(ts_uid),
            pl.col('reference_y').mean(),
            0
            )
    )
    # .join( statsmodel_valid, how="left",  on=[ts_uid, date_col]    )
    .join(
            direct_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            direct_local_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            chain_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    )

forecast_cols = [
                '''
                'HoltWinters', 'AutoETS', "AutoARIMA",
                 'AutoTheta', 'AutoTBATS', 'AutoCES',
                 'arithmetic_forecast_ensamble',
                 '''
                 "reference_y", "global_direct_y_hat", 
                 "local_direct_y_hat", "global_chain_y_hat"
                 ]

forecast_cols = [
    "reference_y", "global_direct_y_hat", 
    "local_direct_y_hat", "global_chain_y_hat"
]

metrics_output_df = []
for col in forecast_cols:
    metrics_output_df.append(
        display_metrics(
        baseline[y].to_numpy(), 
        baseline[col].fill_null(0).to_numpy(),
        name=col
        ).transpose()
    )

allmet = pd.concat(metrics_output_df, axis=1)
allmet.columns = allmet.iloc[0, :]
display(allmet)


"""
with log :

fname	baseline	lgb	lgb_chains
fname	baseline	lgb	lgb_chains
rmse	4123.810547	4999.597825	5121.101499
bias	450.242163	1372.905182	1368.364562
mape	2.105475	1.154826	1.152257
wfiab	0.691066	0.650323	0.651813
mae	1359.627197	1538.938512	1532.378134

without log : 

fname	baseline	lgb	lgb_chains
fname	baseline	lgb	lgb_chains
rmse	4139.15247	2512.594178	2364.075077
bias	421.788203	227.739412	235.493706
mape	2.774925	1.806896	1.947053
wfiab	0.68463	0.797759	0.806641
mae	1387.952732	890.065842	850.979446

wo log | w covid 

fname	baseline	lgb_chains
fname	baseline	lgb_chains
rmse	4139.15247	2292.385397
bias	421.788203	229.864649
mape	2.774925	2.033086
wfiab	0.68463	0.80966
mae	1387.952732	837.692923

fname	baseline	lgb	lgb_chains
fname	baseline	lgb	lgb_chains
rmse	4139.15247	2150.431433	2273.18081
bias	421.788203	101.213962	159.685819
mape	2.774925	2.723164	2.632403
wfiab	0.68463	0.808726	0.807924
mae	1387.952732	841.800341	845.329485


fname	baseline	lgb	lgb_chains
fname	baseline	lgb	lgb_chains
rmse	4143.447587	2214.968268	2127.142468
bias	462.428658	70.666715	109.148351
mape	1.934116	1.718658	1.738316
wfiab	0.692717	0.8319	0.834986
mae	1365.34212	746.9146	733.203599

"""



fname,reference_y,global_direct_y_hat,local_direct_y_hat,global_chain_y_hat
fname,reference_y,global_direct_y_hat,local_direct_y_hat,global_chain_y_hat
rmse,4096.972498,2281.123641,13035.708215,2276.585432
bias,428.170286,152.05768,1211.907011,183.988699
mape,1.906594,1.698225,13.245355,1.792233
wfiab,0.710736,0.843646,0.199865,0.841658
mae,1357.987948,737.750658,5088.93654,747.963718
smape,43.227671,24.40726,122.868208,27.516662


'\nwith log :\n\nfname\tbaseline\tlgb\tlgb_chains\nfname\tbaseline\tlgb\tlgb_chains\nrmse\t4123.810547\t4999.597825\t5121.101499\nbias\t450.242163\t1372.905182\t1368.364562\nmape\t2.105475\t1.154826\t1.152257\nwfiab\t0.691066\t0.650323\t0.651813\nmae\t1359.627197\t1538.938512\t1532.378134\n\nwithout log : \n\nfname\tbaseline\tlgb\tlgb_chains\nfname\tbaseline\tlgb\tlgb_chains\nrmse\t4139.15247\t2512.594178\t2364.075077\nbias\t421.788203\t227.739412\t235.493706\nmape\t2.774925\t1.806896\t1.947053\nwfiab\t0.68463\t0.797759\t0.806641\nmae\t1387.952732\t890.065842\t850.979446\n\nwo log | w covid \n\nfname\tbaseline\tlgb_chains\nfname\tbaseline\tlgb_chains\nrmse\t4139.15247\t2292.385397\nbias\t421.788203\t229.864649\nmape\t2.774925\t2.033086\nwfiab\t0.68463\t0.80966\nmae\t1387.952732\t837.692923\n\nfname\tbaseline\tlgb\tlgb_chains\nfname\tbaseline\tlgb\tlgb_chains\nrmse\t4139.15247\t2150.431433\t2273.18081\nbias\t421.788203\t101.213962\t159.685819\nmape\t2.774925\t2.723164\t2.632403\nwfiab\t

In [74]:
baseline.with_columns(
    pl.concat_list(["reference_y", "global_direct_y_hat", "global_chain_y_hat"]).list.mean().alias('ensemble'),
    pl.concat_list(forecast_cols).list.mean().alias('all_ensemble'),
    pl.concat_list(["global_direct_y_hat", "global_chain_y_hat"]).list.mean().alias('lgb_ensemble'),                   
)


date,station,job,ferie,vacances,index,is_train,y,y_copy,min_dt,year,week,month,day,day_of_year,day_of_week,quarter,week_of_month,is_weekend,Jour de l'an,Lundi de Pâques,Fête du Travail,Fête de la Victoire,Ascension,Lundi de Pentecôte,Fête nationale,Assomption,Toussaint,Armistice,Noël,First Lockdown,Second Lockdown,Third Lockdown (Partial),covid_weight,reference_y,job_right,ferie_right,…,ar_y_win4_shift71_rolling_mean_station@day_of_week,ar_y_win4_shift71_rolling_median_station@day_of_week,ar_y_win4_shift71_rolling_std_station@day_of_week,ar_y_win4_shift71_rolling_skew_station@day_of_week,ar_y_win12_shift27_rolling_mean_station@day_of_week,ar_y_win12_shift27_rolling_median_station@day_of_week,ar_y_win12_shift27_rolling_std_station@day_of_week,ar_y_win12_shift27_rolling_skew_station@day_of_week,ar_y_win12_shift71_rolling_mean_station@day_of_week,ar_y_win12_shift71_rolling_median_station@day_of_week,ar_y_win12_shift71_rolling_std_station@day_of_week,ar_y_win12_shift71_rolling_skew_station@day_of_week,ar_y_lag28.0_station@week,ar_y_lag29.0_station@week,ar_y_lag30.0_station@week,ar_y_lag31.0_station@week,ar_y_lag32.0_station@week,ar_y_lag33.0_station@week,ar_y_win4_shift27_rolling_mean_station@week,ar_y_win4_shift27_rolling_median_station@week,ar_y_win4_shift27_rolling_std_station@week,ar_y_win4_shift27_rolling_skew_station@week,ar_y_win4_shift71_rolling_mean_station@week,ar_y_win4_shift71_rolling_median_station@week,ar_y_win4_shift71_rolling_std_station@week,ar_y_win4_shift71_rolling_skew_station@week,ar_y_win12_shift27_rolling_mean_station@week,ar_y_win12_shift27_rolling_median_station@week,ar_y_win12_shift27_rolling_std_station@week,ar_y_win12_shift27_rolling_skew_station@week,ar_y_win12_shift71_rolling_mean_station@week,ar_y_win12_shift71_rolling_median_station@week,ar_y_win12_shift71_rolling_std_station@week,ar_y_win12_shift71_rolling_skew_station@week,global_direct_y_hat,local_direct_y_hat,global_chain_y_hat
date,str,i64,i64,i64,str,bool,i64,i64,datetime[μs],i16,i16,i16,i16,i16,i16,i16,i16,i16,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,str,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,f64,f64
2022-08-01,"""1J7""",1,0,1,"""2022-08-01_1J7""",true,143,143,2015-01-01 00:00:00,2022,31,8,1,213,1,3,1,0,153,-105,-92,-85,-67,-56,-18,14,92,102,146,0.0,0.0,0.0,0.01,31.0,1,0,…,75.0,72.5,53.944416,0.100333,122.333333,125.5,73.741974,-0.162132,127.166667,124.0,64.286056,0.104489,114,35,165,29,146,168,123.75,139.5,65.703247,-0.599997,null,null,null,null,126.0,155.5,58.715028,-0.730361,null,null,null,null,109.911369,5944.997533,114.335355
2022-08-01,"""O2O""",1,0,1,"""2022-08-01_O2O""",true,25,25,2015-01-02 00:00:00,2022,31,8,1,213,1,3,1,0,153,-105,-92,-85,-67,-56,-18,14,92,102,146,0.0,0.0,0.0,0.01,21.0,1,0,…,null,null,null,null,34.25,34.5,11.354815,-1.32925,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,44.821908,911.85959,43.459832
2022-08-01,"""NZV""",1,0,1,"""2022-08-01_NZV""",true,17,17,2015-01-05 00:00:00,2022,31,8,1,213,1,3,1,0,153,-105,-92,-85,-67,-56,-18,14,92,102,146,0.0,0.0,0.0,0.01,10.0,1,0,…,25.5,24.0,7.593857,0.632839,31.083333,32.0,7.633161,0.187612,24.25,24.5,13.798057,-0.224094,16,30,5,10,null,null,16.5,15.5,10.279429,0.331678,null,null,null,null,15.2,15.0,9.364828,0.694429,null,null,null,null,42.110113,107.278727,41.38109
2022-08-01,"""8QR""",1,0,1,"""2022-08-01_8QR""",true,31,31,2015-01-01 00:00:00,2022,31,8,1,213,1,3,1,0,153,-105,-92,-85,-67,-56,-18,14,92,102,146,0.0,0.0,0.0,0.01,26.0,1,0,…,65.0,67.5,14.854853,-0.517823,56.75,57.5,16.415763,-0.16896,60.5,63.0,15.168748,-0.910178,49,40,55,42,36,34,44.0,44.5,10.099505,-0.125541,null,null,null,null,39.083333,40.5,11.453132,-0.547184,null,null,null,null,63.168051,4742.463486,60.529118
2022-08-01,"""UMC""",1,0,1,"""2022-08-01_UMC""",true,73,73,2015-01-01 00:00:00,2022,31,8,1,2

In [ ]:
import matplotlib.pyplot as plt 

uids = baseline['station'].unique()
n = 6
choice = np.random.choice(uids, (n, ))

# Création de la figure et des sous-graphiques
fig, axs = plt.subplots(3, 2, figsize=(20, 16))
axs = axs.ravel()  # Aplatir le tableau 2D des axes pour itération facile

# Tracer les graphiques
for i, uid in enumerate(uids, start=0):
    if i  < n :
        subset_tr = train_data.filter(
            pl.col(ts_uid) == uid, 
            pl.col('date') >= pl.datetime(2021, 1, 1), 
            pl.col('date') < subset['date'].min()
            ).sort(by='date')
        subset = baseline.filter(pl.col(ts_uid) == uid).sort(by='date')
        axs[i].plot(subset_tr["date"], subset_tr['y_copy'], label="train", color="brown", alpha=0.8)
        axs[i].scatter(subset["date"], subset['y'], label="real", color="blue", alpha=1, marker="*")
        for col in list(filter(lambda x : 'y_hat' in x ,forecast_cols)):
            axs[i].scatter(subset["date"], subset[col], label=col, alpha=0.5, marker="x")
        axs[i].axvline(subset['date'].min())
        axs[i].set_title(f'Graph for ID {uid}')
        axs[i].legend()

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()
plt.show()

overall_valid =  baseline.group_by('date').agg([pl.col(col).sum() for col in forecast_cols])
overall_hist =  train_data.filter(
            pl.col('date') >= pl.datetime(2021, 1, 1), 
            pl.col('date') < subset['date'].min()
            ).sort(by='date').group_by('date').agg(pl.col(y).sum())

overall_realise =  train_data.filter(
            pl.col('date') >= subset['date'].min()
            ).sort(by='date').group_by('date').agg(pl.col(y).sum())

plt.figure(figsize=(25, 6))
plt.scatter(
    overall_hist['date'], overall_hist['y'], label="train", marker="o"
)
plt.scatter(
    overall_realise['date'], overall_realise['y'], label="realise", marker="o"
)
for col in forecast_cols:
    plt.scatter(
        overall_valid['date'], overall_valid[col], label=f"{col}_valid", marker="*"
    )
plt.legend()